# deep featrues를 이용해서 이미지 조회 시스템 만들기

## graphlab import하기

graphlab을 사용하기 위해 import합시다.

In [1]:
import graphlab as gl

# CIFAR-10 이미지 데이터 세트 로드하기

우리가 사용할 데이터는 CIFAR-10이라는 데이터 세트로서 컴퓨터 비전 분야에서 많이 사용하는 벤치 마크 데이터 세트입니다. 

https://www.cs.toronto.edu/~kriz/cifar.html


이 데이터 세트는 원래 10개 카테고리의 이미지들로 구성되어 있는데 본 수업을 위해서 4개 카테고리 - 'cat', 'bird'. automobile', 'dog' - 로 줄였고 학습 데이터와 테스트 데이터로 이미 나누어져 있습니다. 이하에서는 단순히 조회 목적으로만 사용하기 때문에 테스트 세트는 사용하지 않고 학습데이터만 사용할 예정입니다.

아래 data를 로드해서 학습데이터로 사용하고자 합니다.

    'image_train_data/'

In [20]:
train_data = gl.SFrame('../data/image_train_data/')

## 미리 생성된 모델을 이용하여 학습 데이터에 대한 deep features를 계산하기

이하의 2줄은 다른 데이터로 미리 학습된 deep learning 모델을 우리 학습 데이터에 적용하여 deep features를 생성하는 코드입니다. (하지만 시간이 너무 오래 걸리기 때문에 이하의 과정은 생략하고 학습 데이터에 미리 저장해 둔 deep_features 컬럼의 데이터를 사용합니다.)

In [21]:
#deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
#image_train['deep_features'] = deep_learning_model.extract_features(image_train)

학습 데이터에서 deep_features 컬럼의 내용을 살펴 봅시다. 이 학습 데이터에 미리 계산된 deep features값이 저장되어 있는 것을 확인할 수 있습니다.

In [23]:
train_data['deep_features'].head()

dtype: array
Rows: 10
[array('d', [0.24287176132202148, 1.0954537391662598, 0.0, 0.39362990856170654, 0.0, 0.0, 11.894915580749512, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5788496136665344, 0.4954667389392853, 2.5141289234161377, 0.0, 1.5180106163024902, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5813961029052734, 0.0, 0.0, 2.595609426498413, 2.7079553604125977, 0.0, 0.0, 0.0, 0.8509902954101562, 0.0, 0.7203489542007446, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2700355052947998, 0.0, 0.0, 0.0, 0.0, 0.08592796325683594, 0.0, 0.7010231018066406, 0.0, 0.0, 0.0, 0.0, 0.024805665016174316, 0.0, 0.0, 0.17549043893814087, 0.0, 0.0, 0.0, 0.0, 0.0, 2.392784595489502, 0.0, 0.0, 4.471865653991699, 0.0, 1.6358323097229004, 0.0, 4.417484760284424, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.4117904901504517, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1247677206993103, 0.0, 0.0, 0.8957164287567139, 0.0, 0.0, 0.3334987759590149, 0.0, 0.0, 0.2078730463981628

# deep features를 이용해서 이미지 조회를 위한 nearest-neighbors 모델 학습하기

이미지를 하나 주면 유사한 이미지들을 찾아 주는 간단한 이미지 조회 시스템을 만들고자 합니다.

이를 위해 아래의 모델을 사용하고자 합니다.

    graphlab.nearest_neighbors.create(학습데이터, label=레이블컬럼이름, features=피쳐리스트)

In [24]:
knn_deep_feature_model = gl.nearest_neighbors.create(train_data, label='id', features=['deep_features'])

Starting brute force nearest neighbors model training.

# 학습된 모델을 이용해서 유사한 이미지 찾아 보기


학습된 모델을 이용해서 학습 데이터의 19번째 이미지와 비슷한 이미지들을 찾아 봅시다. 

In [25]:
# 먼저 노트북에 캔버스를 고정합시다.
gl.canvas.set_target('ipynb')

해당 이미지를 찾아서 이미지를 출력해 보세요.

In [26]:
train_data['image'][18:19].show()
train_data['id'][18:19]

dtype: int
Rows: 1
[384]

어떤 이미지인가요?

## Quiz

이 이미지와 유사한 이미지들을 찾아 봅시다. 아래 구문을 사용하면 됩니다.
    
    model.query(SFrame데이터)

In [32]:
knn_deep_feature_model.query(train_data[18:19])

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 35.562ms     |

| Done         |         | 100         | 409.981ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,384,0.0,1
0,6910,36.9403137951,2
0,39777,38.4634888975,3
0,36870,39.7559623119,4
0,41734,39.7866014148,5


## Quiz

자동화를 위해서 위의 쿼리 결과를 화면에 바로 이미지로 출력하는 함수를 만들고자 합니다. 어떻게 하면 될까요?

- Query의 결과물이 함수의 인자로 사용됩니다.
- Query의 결과물에서 특정 이미지를 지칭하는 구분자가 무엇인지를 파악합니다.
- 해당 구분자를 이용해서 원하는 이미지들을 학습 데이터에서 골라내야 합니다.
- 골라낸 데이터들의 이미지를 화면에 출력합니다.

아래 메소드를 이용하면 SFrame에서 원하는 값을 가진 데이터만을 추려낸 SFrame을 얻을 수 있습니다.

    SFrame.filter_by(값의리스트, 컬럼이름)

In [29]:
def get_images_from_ids(query_result):
    return train_data.filter_by(query_result['reference_label'],'id')

In [33]:
get_images_from_ids(knn_deep_feature_model.query(train_data[18:19]))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 19.425ms     |

| Done         |         | 100         | 358.252ms    |

+--------------+---------+-------------+--------------+

주어진 이미지와 비슷한 사진들을 찾았나요? 이 정도 코드로 이런 결과가 나온다니 정말 놀랍지 않나요?

## 자동차와 유사한 이미지를 찾아 보기


학습 데이터의 9번 이미지를 이용해서 비슷한 이미지를 찾아 보세요. 위에서 만든 함수를 이용해서 유사한 이미지를 찾아서 출력해 봅시다.

In [40]:
car = train_data[8:9]
car['image'].show()

In [42]:
get_images_from_ids(knn_deep_feature_model.query(car))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 20.302ms     |

| Done         |         | 100         | 278.522ms    |

+--------------+---------+-------------+--------------+

## lambda 함수를 이용해서 근접 이웃 이미지들을 화면에 출력해 봅시다.

학습데이터의 행수 (몇번째 데이터인지)를 입력하면 해당 이미지의 근접 이웃 이미지를 출력하는 함수를 lambda로 만들어 봅시다. 위에서 만든 함수를 이용하면 쉽게 만들 수 있습니다.

    함수이름 = lambda 변수명: 변수명이들어간구문

In [43]:
show_neighbors = lambda i : get_images_from_ids(knn_deep_feature_model.query(train_data[i:i+1]))['image'].show()

## Quiz

학습 데이터의 27번째 이미지와 유사한 이미지를 찾아 보세요

In [44]:
show_neighbors(26)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 29.77ms      |

| Done         |         | 100         | 272.068ms    |

+--------------+---------+-------------+--------------+

In [45]:
show_neighbors(1500)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 22.167ms     |

| Done         |         | 100         | 298.143ms    |

+--------------+---------+-------------+--------------+